# NLP Assignment $2$

*Neelay Upadhyaya __18210053__*

*M.Tech CSE*

#### Download any of these text books from Project Gutenberg 

Text used: Sherlock Holmes: The Adventures of Sherlock Holmes

In [ ]:
from nltk.tokenize import sent_tokenize
import random

corpus = open('sherlock.txt', 'r')


sentences = sent_tokenize(corpus.read())

# cleaning and pre-processing data.
sentences = ['<s> ' + word.strip().replace('\n', ' ').lower() + ' </s>' for word in sentences]

#random.shuffle(sentences)
training_index = int(len(sentences) * 0.8)
sentences_training = sentences[:training_index]
sentences_test = sentences[training_index:]

print('Total sentences:', len(sentences))
print('Training sentences:', len(sentences_training))
print('Training sentences:', len(sentences_test))


In [ ]:
# This class has all the functionalities required for n-grams.
import math
import numpy as np

class NGrams:
    def __init__(self, sentences=[]):
        self.sentences = sentences
        self.master_dictionary = {}
        self.mle_dictionary = {}
        
        
    def __ngrams(self, n):
        dictionary = {}
        for sentence in self.sentences:
            words = [word for word in sentence.split()]
            for i in range(0, len(words) - n + 1):
                ngram = ' '.join(words[i : i + n])
                frequency = dictionary.get(ngram, 0)
                dictionary[ngram] = frequency + 1
        
        self.master_dictionary[n] = dictionary
        return self.master_dictionary[n]
    
    
    def unigrams(self):
        return self.__ngrams(1)
    
    
    def bigrams(self):
        return self.__ngrams(2)
    
    
    def trigrams(self):
        return self.__ngrams(3)
    
    
    def quadgrams(self):
        return self.__ngrams(4)
    
    
    def __compute_ngram_mle(self, n):
        if n == 1:  
            denominator = sum(self.master_dictionary[n].values())
        
        dictionary = {}
        for ngram_word, freq in self.master_dictionary[n].items():
            words = ngram_word.split()
            
            if n != 1:
                denominator = self.master_dictionary[n - 1][' '.join(words[:-1])]

            probability = freq / denominator
            dictionary[ngram_word] = probability
            
        self.mle_dictionary[n] = dictionary
        return self.mle_dictionary[n]
    
        
    def compute_unigram_mle(self):
        return self.__compute_ngram_mle(1)    
    
    
    def compute_bigram_mle(self):
        return self.__compute_ngram_mle(2)
        
        
    def compute_trigram_mle(self):
        return self.__compute_ngram_mle(3)
    
    
    def compute_quadgram_mle(self):
        return self.__compute_ngram_mle(4)
        
    
    def find_probability_of_sentence(self, sentence, n):
        sentence = '<s> ' + sentence + ' </s>'
        words = sentence.split()
        probability = 0
        
        for i in range(len(words) - n + 1):
            ngram = ' '.join(words[i : i + n])
            print(ngram)
            mle = self.mle_dictionary[n].get(ngram, 0)
            print(mle)
            if mle == 0:
                return 'Cannot find the probability of the sentence, as the corpus has no "' + ngram + '"'
            probability += math.log(mle)
            
        return probability
            
    def __random_next(self, n, given_word):
        current_dictionary = self.master_dictionary[n]
        
        words = list(current_dictionary.keys())
        given_candidates = [word for word in words if word.startswith(given_word)]
        
        if len(given_candidates) == 0:
            return '</s>'
        
        candidate_probabilites = []
        for i in range(len(given_candidates)):
            candidate_probabilites.append(self.mle_dictionary[n].get(given_candidates[i], 0))
        
        normalization_factor = sum(candidate_probabilites)
        
        for index, probability in enumerate(candidate_probabilites):
            candidate_probabilites[index] = probability / normalization_factor
        
        # print(candidate_probabilites[:20])
        outcomes = list(np.random.multinomial(100, candidate_probabilites))
        max_value = max(outcomes)
        max_index = outcomes.index(max_value)
        
        return ' '.join(given_candidates[max_index].split()[1:])
            
    def generate_sentence(self, model):
        sentence = '<s> '
        next_word = self.__random_next(model, '<s>')
        
        sentence += next_word + ' '
    
        while(not next_word.endswith('</s>')):
            next_word = next_word.split()
            next_word = self.__random_next(model, next_word[-1])
            sentence += next_word + ' '
        
        return sentence.strip()

#ngram = NGrams(["I am Sam", "Sam I am", "I do not like green eggs and ham"])
ngram = NGrams(sentences_training)


#### N-grams in corpus

In [ ]:
unigrams = ngram.unigrams()
bigrams = ngram.bigrams()
trigrams = ngram.trigrams()
quadgrams = ngram.quadgrams()

for key, value in bigrams.items():
    print(key, " ", value)

#### MLE for unigram, bigram, trigrams and quadgrams.

In [ ]:
unigram_mle = ngram.compute_unigram_mle()
bigram_mle = ngram.compute_bigram_mle()
trigram_mle = ngram.compute_trigram_mle()
quadgram_mle = ngram.compute_quadgram_mle()

for key, value in bigram_mle.items():
    print(key, " ", value)

#### Max possible N-grams Vs Actual N-grams

In [ ]:
v = len(unigrams)

print('Total types ', v)

print('Total possible bigrams count: ', v ** 2)
print('Total possible trigrams count: ', v ** 3)
print('Total possible quadgrams count: ', v ** 4)

print('Total actual bigrams count:', len(bigrams))
print('Total actual trigrams count:', len(trigrams))
print('Total actual quadgrams count:', len(quadgrams))

In [ ]:
def validate_model_type(model):
    return model.isnumeric() and int(model) in [1, 2, 3, 4]

def validate_number_of_sentences(number):
    return number.isnumeric() and 2 <= int(number) <= 10
    

#### Sentence Generation according to the model 

In [ ]:
model_type = input("Enter a model for which you want to generate sentences: \n 1 for unigram \n 2 for bigram \n 3 for trigram \n 4 for quadgram \n")

number_of_sentences = input("Enter how many sentences you want? Enter between 2-10 inclusive \n")

if validate_model_type(model_type) and validate_number_of_sentences(number_of_sentences):
    for i in range(int(number_of_sentences)):
        print(ngram.generate_sentence(int(model_type)))
else:
    print("You did not enter a valid input.")

#### Calculating the probability of the sentence

In [ ]:
input_sentence = input("Enter a sentence for which you want to find probability: \n")

model_type = input("Enter a model for which you want to generate sentences: \n 1 for unigram \n 2 for bigram \n 3 for trigram \n 4 for quadgram \n")

if validate_model_type(model_type):
    probability = ngram.find_probability_of_sentence(input_sentence.lower(), int(model_type))
    print(probability)
    
else:
    print("You did not enter a valid input.")

In [ ]:
# This class provides the required functionalities for smoothing.
class Smoothing:
    
    def add_one(self, bigrams, unigrams):
        new_bigrams = {}
        new_bigram_mle = {}
        v = len(unigrams)
        for ngram, freq in bigrams.items():
            words = ngram.split()
            denominator = unigrams.get(words[0], 0) + v
            probability = (freq + 1) / denominator
            new_bigram_mle[ngram] = probability
            new_bigrams[ngram] = probability * unigrams.get(words[0], 0)
            
        return new_bigrams, new_bigram_mle
    
    def good_turing(self, unigrams):
        frequency_of_frequency = {}
        for value in unigrams.values():
            frequency = frequency_of_frequency.get(value, 0)
            if(frequency == 0):
                frequency_of_frequency[value] = 1
            else: 
                frequency_of_frequency[value] += 1
                
        v = len(unigrams)
        
        good_turing_counts = {}
        good_turing_counts[0] = frequency_of_frequency.get(1) / v
        
        for i in range(1, 11):
            good_turing_counts[i] = (i + 1) * frequency_of_frequency.get(i + 1, 0) / frequency_of_frequency.get(i, 0)
        
        return good_turing_counts
    
    def good_turing_smoothing(self, d):
        new_bigrams = {}
        new_bigram_mle = {}
        v = len(unigrams)
        for ngram, freq in bigrams.items():
            words = ngram.split()
            denominator = unigrams.get(words[0], 0) + v
            probability = (freq - d) / denominator
            new_bigram_mle[ngram] = probability
            new_bigrams[ngram] = probability * unigrams.get(words[0], 0)
            
        return new_bigrams, new_bigram_mle
    
    def get_good_turing_discount_value(self, good_turing_counts):
        differences = []
        for key, value in good_turing_counts.items():
            differences.append(abs(key - value))
        
        #print(differences)
        return sum(differences) / float(len(differences))
        
smoothing = Smoothing()
        

#### Implementing add-1 smoothing

In [ ]:
add_one_smoothing = smoothing.add_one(bigrams, unigrams)
new_bigrams_add_one = add_one_smoothing[0]
new_bigrams_mle = add_one_smoothing[1]

for k, v in new_bigrams_mle.items():
    print(k, " ", v)

#### Examples where add-1 smoothing causes drastic changes.

In [ ]:
print('Original bigram count for "for a":',bigrams['for a'])
print('New bigram count for "for a":', new_bigrams_add_one['for a'])

print('Original bigram count for "when he":', bigrams['when he'])
print('New bigram count for "when he":',new_bigrams_add_one['when he'])

print('Original bigram count for "of her":',bigrams['of her'])
print('New bigram count for "of her":',new_bigrams_add_one['of her'])

As we can see above, there are counts which have decreased by a factor of 8-20. 

The reason for this drastic change is that a lot of probability mass is moved to the bigrams with count 0

#### Discount value by implementing Good-Turing smoothing

In [ ]:
good_turing_counts = smoothing.good_turing(unigrams)
d = smoothing.get_good_turing_discount_value(good_turing_counts)
print('Constant Discount value(d): ', d)


good_turing_smoothing = smoothing.good_turing_smoothing(d)

#### Perplexity for add-1 and Good-Turing smoothing on Test dataset

In [ ]:
class Perplexity:
    
    def __init__(self, sentences):
        self.sentences = sentences
        
        
    def add_one(self, bigrams, unigrams):
        smoothed_bigrams = {}
        for sentence in self.sentences:
            words = sentence.split()
            for index in range(len(words) - 1):
                ngram = words[index] + ' ' + words[index + 1]
                #print(ngram)
                if ngram not in bigrams:
                    denominator = unigrams.get(words[index], 0) + v
                    probability = (bigrams.get(ngram, 0) + 1) / denominator
                    smoothed_bigrams[ngram] = probability
                    
        return smoothed_bigrams
    
    def good_turing(self, bigrams, unigrams, d):
        smoothed_bigrams = {}
        for sentence in self.sentence:
            words = sentence.split()
            for index in range(len(words) - 1):
                ngram = words[index] + ' ' + words[index + 1]
                #print(ngram)
                if ngram not in bigrams:
                    denominator = unigrams.get(words[index], 0) + v
                    probability = (bigrams.get(ngram, 0) - d) / denominator
                    smoothed_bigrams[ngram] = probability
                    
        return smoothed_bigrams
            
    
    def add_one_perplexity(self, add_one_mle):
        probability = 1
        num_words = sum([len(sentence.split()) for sentence in self.sentences])
        num_words = 2
        for sentence in self.sentences:
            words = sentence.split()
            for i in range(len(words) - 1):
                ngram = ' '.join(words[i : i + 2])
                mle = add_one_mle.get(ngram, 1)
                probability *= (1.0 / mle)
                probability = (probability ** (1.0 / num_words))
                
        return probability
    
    def good_turing_perplexity(self, good_turing_mle):
        probability = 1
        num_words = sum([len(sentence.split()) for sentence in self.sentences])
        for sentence in self.sentences:
            words = sentence.split()
            for i in range(len(words) - 1):
                ngram = ' '.join(words[i : i + 2])
                mle = good_turing_mle.get(ngram, 1)
                probability *= (1.0 / mle)
                probability = (probability ** (1.0 / num_words))
                
        return probability

perplexity = Perplexity(sentences_test)
pp_one = perplexity.add_one_perplexity(new_bigrams_mle)
print("Perplexity for add-1:",pp_one)
pp_good_turing = perplexity.good_turing_perplexity(good_turing_smoothing[1])
print("Perplexity for Good-Turing:",pp_good_turing)

From the above numbers, PP(add-1) > PP(Good-Turing). Thus Good-Turing performs better.